# Orojenesis Artifact - Single-Ensum

The ipython notebook file contains calls to Orojenesis to reproduce results in the ISCA'24 *"Mind the Gap: Attainable Data Movement and Operational Intensity Bounds for Tensor Algorithms"* paper. 

## 0.  Setup Software Dependencies 
 Please first run install.sh to install software dependencies.

In [ ]:
import os
if "TIMELOOP_BASE_PATH" not in os.environ:
    timeloop_path = input("Please specify the path to Timeloop repo (default: " +  os.getcwd() + "/timeloop):" ) or os.getcwd() + "/timeloop"
    os.environ["TIMELOOP_BASE_PATH"] = timeloop_path
os.environ["TIMELOOP_ENABLE_FIRST_READ_ELISION"] = "1"
print("Path to timeloop repo: ", os.environ["TIMELOOP_BASE_PATH"])
import src.utils as utils
import src.plots as plots
import pathlib

## 1. Generate Single-Einsum Bounds
The *Orojenesis* flow has been integrated into Timeloop mapper to log the statistics of all pareto-optimal mappings. In order to turn the *orojenesis* option on, set `log-oaves` to `True` in the Timeloop mapper configuration file, e.g., `./configs/single-einsum/conv_mapper.yaml`. The flow generates the `timeloop-mapper.oaves.csv` file under the output directory and we need to run a post processing script `./src/oaves_process_data.py` to merge the pareto-optimal mappings from different mapper threads. 

We first specify the output directory for the results, path to the speeder architecture, and the mapper constraints for convolution. 

In [ ]:
output_dir = pathlib.Path('./outputs/single-einsum')
arch_yaml = pathlib.Path('./configs/single-einsum/arch.yaml')
mapper_yaml = pathlib.Path('./configs/single-einsum/conv_mapper.yaml')

### Run Orojenesis search to generate bounds  (Estimated Runtime: 8 hours) 
This artifact  takes **several hour** to finish. The actual runtime may vary depending on your processor's speed and core count. Once the bounds are generated, the following code can directly load the generated data without initiating a rerun.  

In [ ]:
force_rerun = False
# Fig.1 Motivation figure 
fig1_prob = utils.Conv(P=16384, C=1024, K=1024)
utils.GenerateBound(fig1_prob, output_dir, arch_yaml, mapper_yaml, force_rerun=force_rerun)

# Fig.10 Impact of GEMM sizes. 
fig10_probs = []
prob=[2048,2048,2048]
for i in range(-1, 6):
    if i >= 0:
        idx = 2 - (i % 3)
        prob[idx] *= 2
    fig10_prob = utils.Conv(P=prob[0], C=prob[1], K=prob[2])
    utils.GenerateBound(fig10_prob, output_dir, arch_yaml, mapper_yaml, keep_one_best_entry_across_buf=True, force_rerun=force_rerun)
    fig10_probs.append(fig10_prob)

# Fig.11 Maximal effectualbuffer ratio over total operand size. 
fig11_probs = []
factors = [0.5, 1, 2]
for m in factors:
    for k in factors:
        for n in factors: 
            prob = [int(4096 * m), int(4096 * k), int(4096 * n)]
            fig11_prob = utils.Conv(P=prob[0], C=prob[1], K=prob[2])
            utils.GenerateBound(fig11_prob, output_dir, arch_yaml, mapper_yaml, keep_one_best_entry_across_buf=True, force_rerun=force_rerun)
            fig11_probs.append(fig11_prob)

# Fig.12 Impact of conv configs. 
fig12_probs = []
probs = ['1_1_16_16_64_64_1_1_1_1_1', '3_3_16_16_64_64_1_1_1_1_1', '5_5_16_16_64_64_1_1_1_1_1', '3_3_16_16_64_64_1_2_2_1_1', '3_3_16_16_64_64_1_1_1_2_2']
for prob in probs:
    factors = [int(string) for string in prob.split('_')]
    fig12_prob = utils.Conv(*factors)
    utils.GenerateBound(fig12_prob, output_dir, arch_yaml, mapper_yaml, \
                        keep_one_best_entry_across_buf=True, force_rerun=force_rerun)
    fig12_probs.append(fig12_prob)

# Fig.13 Impact of BMM heads with fixed total OPs. bmm_QK[hlf,hfl->hll]
# This experiment takes roughly 5 mins to finish.
fig13_probs = []
for i in range(8):
    num_heads = int(2**i)
    K = 4096 // num_heads
    fig13_prob = utils.GBMM(M=4096, K=K, N=4096, H=num_heads)
    utils.GenerateBound(fig13_prob, output_dir, arch_yaml, pathlib.Path('./configs/single-einsum/gbmm_mapper.yaml'), \
                        keep_one_best_entry_across_buf=True, force_rerun=force_rerun)
    fig13_probs.append(fig13_prob)

# Fig.14 Impact of BMM groups. Note that postprocessing is required to obtain the total OPs in Timeloop
# This experiment takes roughly 6.5 hours to finish.
fig14_probs = []
for i in range(6):
    num_groups = int(2**i)
    fig14_prob = utils.GBMM(M=4096, K=128, N=4096, H=32, G=num_groups)
    utils.GenerateBound(fig14_prob, output_dir, arch_yaml, pathlib.Path('./configs/single-einsum/gbmm_mapper.yaml'), \
                        keep_one_best_entry_across_buf=True, force_rerun=force_rerun)
    fig14_probs.append(fig14_prob)

## 2. Plot Single-Einsum Bounds
We save the generated figures under `fig_dir`. 

In [ ]:
fig_dir = pathlib.Path('./figs')
fig_dir.mkdir(parents=True, exist_ok=True)

## Fig.1: Backing-store accesses bound for 16384x1024x1024 GEMM

In [ ]:
stats_files = utils.get_stats_files(output_dir, [fig1_prob])
dfs = utils.get_dfs(stats_files, get_opt=True)
y_end_value=10**8
plots.plot_dfs(dfs, logy=True, logx=True, shape_name="fig1", probs=[fig1_prob], motivation=True, plot_min=True, plot_buf=False, plot_all_mappings=True, xlim=(0.5, y_end_value), ylim=(None, 10**10), y_end_value=10**8)

## Fig.10: Backing-store accesses and OI bounds for various GEMM shapes.

In [ ]:
legends = ['2048_2048_2048','2048_2048_4096', '2048_4096_4096', '4096_4096_4096', '4096_4096_8192', '4096_8192_8192', '8192_8192_8192']
stats_files = utils.get_stats_files(output_dir, fig10_probs)    
dfs = utils.get_dfs(stats_files, get_opt=True)
ax = plots.plot_dfs(dfs, legends=legends, dpi=300, logy=True, logx=True, shape_name="fig10", figsize=(2.5, 2.5),  xlim=(10**4, 1*10**8), ylim=(0, 10*10**9), y_end_value=2*10**8, legend_fontsize=7.5)

plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=False, metric="Op_Intensity", shape_name="fig10",  figsize=(2.5,2.5),  xlim=(10**4, 2*10**8), ylim=(0, 6*10**3), y_end_value=2*10**8, legend_fontsize=7.5)

## Fig.11: Maximal effectual buffer ratio over total operand size for various GEMMs.

In [ ]:
legends = []
factors = [0.5, 1, 2]
for m in factors:
    for k in factors:
        for n in factors: 
            M = int(4096 * m)
            N = int(4096 * k)
            K = int(4096 * n)
            legends.append(f'{M}_{K}_{N}')    
plots.plot_bar_ratios(output_dir, fig11_probs, legends, fig_name='fig11', figsize=(6, 6), sort_ratio=True)

## Fig.12: Backing-store accesses and OI bounds for various convolution configurations.

In [ ]:
legends = ['1x1 conv', '3x3 conv', '5x5 conv', '3x3 conv\nstride 2', '3x3 conv\ndilation 2']
stats_files = utils.get_stats_files(output_dir, fig12_probs)    
dfs = utils.get_dfs(stats_files, get_opt=True)
y_end_value = 1*10**5
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, shape_name="fig12", figsize=(2.5,2.5),  xlim=(None,y_end_value), ylim=(0, 4*10**7), y_end_value=y_end_value, legend_fontsize=9)
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, metric="Op_Intensity", shape_name="fig12",  figsize=(2.5,2.5),  xlim=(None,y_end_value), ylim=(0, 400), y_end_value=y_end_value, legend_fontsize=9)

## Fig.13: Backing-store accesses and OI bounds for BMMs with different number of heads but identical OPs.

In [ ]:
legends = []
for i in range(8):
    num_heads = int(2**i)
    K = 4096 // num_heads
    K_value = f'{K//1000}k' if K > 1000 else K
    legends.append(f'H={num_heads}, K={K_value}')

stats_files = utils.get_stats_files(output_dir, fig13_probs)        
dfs = utils.get_dfs(stats_files, get_opt=True)
y_end_value = 2*10**8
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, shape_name="fig13", figsize=(2.5,2.5),  xlim=(10**3,y_end_value), ylim=(0, 4*10**9), y_end_value=y_end_value, legend_fontsize=8)
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, metric="Op_Intensity", shape_name="fig13",  figsize=(2.5,2.5),  xlim=(10**3,y_end_value), ylim=(0, 3*10**3), y_end_value=y_end_value, legend_fontsize=8)

## Fig.14: Backing-store accesses and OI bounds for Grouped BMMs with different number of groups but identical OPs.

In [ ]:
legends = []
for i in range(6):
    num_groups = int(2**i) 
    if num_groups == 1:
        legends.append(f'{num_groups} group')
    else:
        legends.append(f'{num_groups} groups')
    
    
stats_files = utils.get_stats_files(output_dir, fig14_probs)        
dfs = utils.get_dfs(stats_files, get_opt=True)
total_compute = fig14_probs[0].get_compute_size()
for i, df in enumerate(dfs):
    dfs[i]['Op_Intensity'] = total_compute/dfs[i]['DRAM_Accesses']
y_end_value = 2*10**8
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, shape_name="QK_sweep_groups", figsize=(2.5,2.5),  xlim=(None,y_end_value), ylim=(0, 1.5*10**12), y_end_value=y_end_value, legend_fontsize=8)
plots.plot_dfs(dfs, legends=legends, dpi=300, logy=False, logx=True, metric="Op_Intensity", shape_name="QK_sweep_groups",  figsize=(2.5,2.5),  xlim=(None,y_end_value), ylim=(1, 140), y_end_value=y_end_value,  legend_fontsize=8)

## 3. Validation on GPUs and Simba 
We validate the Orojenesis curve for a 4kx4kx4k GEMM on four NVIDIA GPUs and on a model of the Simba accelerator. 

## Fig.24a: Measured DRAM accesses on GPUs with different L2 sizes vs. Orojenesis Bounds
This figure shows the DRAM accesses measured on NVIDIA GA100 GPUs with various L2 sizes  

[OPTIONAL] Below is the instruction for getting the GPU dram access count using CULTASS.   
1. Download and install the [CUTLASS library](https://github.com/NVIDIA/cutlass/tree/cutlass-3.5.0) following the [Quick Start Guide](https://github.com/NVIDIA/cutlass/blob/cutlass-3.5.0/media/docs/quickstart.md). Set the CUTLASS_NVCC_ARCH=80 to compile the library for A100 GPUs.  
2. Download and install the GPU profiling tool Nsight Compute following the instructions (here)[https://developer.nvidia.com/tools-overview/nsight-compute/get-started].
3. Navigate to the build directory of cutlass.  
```
cd <path-to-cutlass>/build
```
4. Run the following command to gather the DRAM accesses count for running fp32 4kx4kx4k on SMs.  
```
ncu  --cache-control all  --metrics dram__bytes_read.sum,dram__bytes_write.sum ./tools/profiler/cutlass_profiler --kernels=sgemm --gemm_kind=universal --m=4096 --n=4096 --k=4096 --A=f32:column --B=f32:column --C=f32:column --D=f32:column --verification-enabled=0   --profiling-enabled=1   --profiling-iterations=1 --warmup-iterations=0
```
5. Run the following command to gather the DRAM accesses count for running fp32 4kx4kx4k on tensor cores.  

```
ncu  --cache-control all  --metrics dram__bytes_read.sum,dram__bytes_write.sum \
./tools/profiler/cutlass_profiler \
    --kernels=cutlass_tensorop_s1688bf16gemm_256x128_16x3_nn_align4 \
    --operation=gemm \
    --op_class=tensorop \
    --gemm_kind=universal \
    --m=4096 --n=4096 --k=4096 \
    --A=f32:column --B=f32:column --C=f32:column --D=f32:column --accum=f32 \
    --verification-enabled=0   --profiling-enabled=1   --profiling-iterations=1 --warmup-iterations=0 \
    --mode=trace
```

6. Repeat the process on different GPUs. 

In this notebook, we provide the pregenerated GPU accesses to compare to the *Orojenesis* bound.

In [ ]:
prob = utils.Conv(P=4096, C=4096, K=4096)
utils.GenerateBound(prob, output_dir, arch_yaml, mapper_yaml, \
                    keep_one_best_entry_across_buf=True, force_rerun=False)
stats_files = utils.get_stats_files(output_dir, [prob])    
dfs = utils.get_dfs(stats_files, get_opt=True)
y_end_value = 2*10**8
gpu_data = {
    'simt': [[2*2**20, 24*2**20, 40*2**20, 50*2**20], [2.69*2**30, 650*2**20, 533*2**20, 373.39*2**20]],
    'tensor': [[2*2**20, 24*2**20, 40*2**20, 50*2**20], [1.58*2**30, 561.51*2**20, 411.06*2**20,373.33*2**20]],
}
# df = dfs[0]
# df.index=df.index*4
# df['DRAM_Accesses'] = df['DRAM_Accesses']*4
# print(df)
ax = plots.plot_dfs(dfs, legends=['4k_4k_4k'], dpi=300, logy=False, logx=True, shape_name="MM_same_flops", figsize=(2.5,2.5),  xlim=(10**6,y_end_value), ylim=(0, 4*10**9), y_end_value=y_end_value, plot_gpu_data=True, gpu_data=gpu_data, coefficient=4, legend_fontsize=8)


## Fig.24b: Measured DRAM accesses on Simba accelerator vs. Orojenesis Bounds 

In [ ]:
val_output_dir = pathlib.Path('./outputs/validation')
simba_arch_yaml = pathlib.Path('./configs/simba/simba-chip.yaml')
simba_mapper_yaml = pathlib.Path('./configs/simba/mapper.yaml')

prob = utils.Conv(P=4096, C=4096, K=4096)

# It takes roughly 3 minute to generate 2000 valid mappings for each simba config. 
buf_KBs = [0.125, 1, 8, 64, 512]
simba_csvs = []
force_rerun = False
for buf_KB in buf_KBs:
    arch = utils.parse_yaml(simba_arch_yaml)
    for idx, mem in enumerate(arch['arch']['storage']):
        if mem['name'] == 'GlobalBuffer':
            if buf_KB < 1:
                del mem['sizeKB']
                mem['entries'] = int(1024 * buf_KB)
            else:
                mem['sizeKB'] = buf_KB

    output_subdir = val_output_dir / 'simba' / pathlib.Path(f'output_{buf_KB}')
    output_subdir.mkdir(exist_ok=True, parents=True)
    new_arch_yaml =  output_subdir / 'arch.yaml'
    utils.store_yaml(new_arch_yaml, arch)
    utils.GenerateBound(prob, output_subdir, new_arch_yaml, simba_mapper_yaml, keep_one_best_entry_across_buf=False, force_rerun=force_rerun)
    stats_file = utils.get_stats_files(output_subdir, [prob], pareto_optimal=False)[0] 
    simba_csvs.append(stats_file)

y_end_value = 2*10**8
simba_dfs = utils.get_dfs(simba_csvs, scales=None, get_opt=False, get_mapping=False)
stats_files = utils.get_stats_files(output_dir, [prob])    
dfs = utils.get_dfs(stats_files, get_opt=True)
ax = plots.plot_dfs(dfs, legends=['4k_4k_4k'], dpi=300, logy=False, logx=True, shape_name="simba_val", figsize=(2.5,2.5),  xlim=(0,12**9), ylim=(0, 100*10**9), y_end_value=y_end_value, simba_dfs=simba_dfs, legend_fontsize=8)